In [2]:
##This repo contains the ipython notebooks implementing CNN, RNN and HAN for text classification.
##https://github.com/jatana-research/Text-Classification/blob/master/README.md
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
#os.environ['KERAS_BACKEND']='theano' # Why theano why not
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline
print('import done')

Using TensorFlow backend.


import done


In [3]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 200
VALIDATION_SPLIT = 0.3

In [4]:
# reading data
data_file ='C:/Users/khmar/Desktop/ISSUE/dataset/CSV/data_ameliorate/data.csv'
df = pd.read_csv(data_file,delimiter=';',encoding='latin-1')
df = df.dropna()
df = df.reset_index(drop=True)
print('Shape of dataset ',df.shape)
print(df.columns)
print('No. of unique classes',len(set(df['label'])))

Shape of dataset  (4055, 2)
Index(['text', 'label'], dtype='object')
No. of unique classes 2


In [5]:
macronum=sorted(set(df['label']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

df['label']=df['label'].apply(fun)

In [6]:
texts = []
labels = []


for idx in range(df.text.shape[0]):
    text = BeautifulSoup(df.text[idx])
    texts.append(str(text.get_text().encode()))

for idx in df['label']:
    labels.append(idx)

In [7]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Number of Unique Tokens',len(word_index))

Number of Unique Tokens 5560


In [8]:
tok_file = 'token_CNN_GLOVE.200d_DATA.sav'
pickle.dump(tokenizer, open(tok_file, 'wb'))

In [9]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of Data Tensor:', data.shape)
print('Shape of Label Tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Shape of Data Tensor: (4055, 1000)
Shape of Label Tensor: (4055, 2)


In [10]:
embeddings_index = {}
f = open('C:/Users/khmar/Desktop/GLOVE/glove.6B.200d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.'% len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [11]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,trainable=True)

In [12]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(len(macronum), activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("Simplified convolutional neural network")
model.summary()
cp=ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)

Instructions for updating:
Colocations handled automatically by placer.
Simplified convolutional neural network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 200)         1112200   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          128128    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
______________________________

In [13]:
history=model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=2, batch_size=2,callbacks=[cp])

Instructions for updating:
Use tf.cast instead.
Train on 2839 samples, validate on 1216 samples
Epoch 1/2
2839/2839 [==============================] - 117s 41ms/step - loss: 0.6283 - acc: 0.6375 - val_loss: 0.4413 - val_acc: 0.7985

Epoch 00001: val_acc improved from -inf to 0.79852, saving model to model_cnn.hdf5
Epoch 2/2
2839/2839 [==============================] - 107s 38ms/step - loss: 0.4490 - acc: 0.8225 - val_loss: 0.4318 - val_acc: 0.7952

Epoch 00002: val_acc did not improve from 0.79852


In [14]:
scores= model.evaluate(x_val, y_val, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[0], scores[0] * 100))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

loss: 43.18%
acc: 79.52%


In [15]:
model.save("model_CNN_GLOVE_200d_without_text_preprocessing")